# 量子コンピュータをモンテカルロサンプラーとして使う

ここでは、今までとは異なる視点で量子コンピュータを使うことを考えてみます。特に、量子状態を測定することで所望の確率分布をサンプリングするためのアルゴリズムを紹介して、その実装を試みます。

```{contents} 目次
---
local: true
---
```

## はじめに <a id='introduction'></a>

量子コンピュータが。。。

## 確率分布のサンプリング <a id='sampling'></a>

## 量子サンプリングアルゴリズム <a id='algorithm'></a>

まず$[0,\:1]$の範囲の値をとる$N$個の実数$p$を考えて、この実数の2乗が確率を与えると考えてみます。つまり、0から$N-1$までの$N$個の整数を$n=\lceil\log_2N\rceil$個の量子ビットの計算基底$\{|l\rangle\;|\;l=0,1,\cdots,N-1\}$で表現するとした場合、
$\{p_l^2\in[0,\;1]\:|\:l=0,1,\cdots,N-1\}$の確率分布を生成する$n$量子ビット状態を生成することがゴールです。言い換えれば、ある初期状態から出発して、最後に状態

$$
|\psi\rangle=\frac1A\sum_{l=0}^{N-1}p_l|l\rangle
$$

（ここで$A=\sqrt{\sum_{l=0}^{N-1}p_l^2}$は確率の総和を1に規格化するための因子）を生成したいわけです。この状態を作ることができれば、その状態を測定することで各計算基底を確率$(p_l/A)^2$で得ることができます。

これは一見簡単そうに見えるかもしれませんが、任意の数を振幅として持つ状態を作ることは単純なゲート操作では実現できないので、実は簡単な問題ではありません。以下に、それをどのように実装していくか基本的な部分から見ていきます。


### 構成要素 <a id='algo_elements'></a>

量子サンプリングを実現するために、「**量子位相推定」**（*Quantum Phase Estimation*, QPE）と呼ばれる手法を使うことにします。QPEの理解には「**量子フーリエ変換**」（*Quantum Fourier Transform*, QFT）の理解が欠かせませんが、QFTについては、この[実習](circuit_from_scratch.ipynb)の問題7もしくは参考文献[1]を参照してください。

QPEが考える問題はどういうものかと言うと、「あるユニタリー演算$U$に対して$U|\psi\rangle=e^{2\pi i\theta}|\psi\rangle$となる固有ベクトル$|\psi\rangle$が与えられるとして、その固有値$e^{2\pi i\theta}$の位相$\theta$を求めることができるか？」という問題です。

#### 1量子ビットの位相推定 <a id='qpe_1qubit'></a>
まず、下図にあるような量子回路を考えてみましょう。

```{image} figs/qpe_1qubit.png
:alt: qpe_1qubit
:class: bg-primary mb-1
:width: 300px
:align: center
```

この場合、量子回路のステップ 1-3での量子状態は以下のようになります。

- ステップ 1 : $\frac{1}{\sqrt{2}}(|0\rangle|\psi\rangle+|1\rangle|\psi\rangle)$
- ステップ 2 : $\frac{1}{\sqrt{2}}(|0\rangle|\psi\rangle+|1\rangle e^{2\pi i\theta}|\psi\rangle)$
- ステップ 3 : $\frac{1}{2}\left[(1+e^{2\pi i\theta})|0\rangle+(1-e^{2\pi i\theta})|1\rangle\right]|\psi\rangle$

この状態で上側の量子ビットを測定すると、$|(1+e^{2\pi i\theta})/2|^2$の確率で0、$|(1-e^{2\pi i\theta})/2|^2$の確率で1を測定するでしょう。つまり、この確率の値から位相$\theta$を求めることができるわけです。
しかし、$\theta$の値が小さい場合、ほぼ100%の確率で0を、ほぼ0％の確率で1を測定することになるため、100%あるいは0%からのずれを精度良く決めるためには多数回の測定が必要になります。

少ない測定回数でも精度良く位相を決める方法は、何か考えられるでしょうか。。

#### $n$量子ビットの位相推定 <a id='qpe_nqubit'></a>
そこで、上側のレジスタを$n$量子ビットに拡張した量子回路（下図）を考えてみましょう。

```{image} figs/qpe_wo_iqft.png
:alt: qpe_wo_iqft
:class: bg-primary mb-1
:width: 500px
:align: center
```

それに応じて、下側のレジスタにも$U$を繰り返し適用することになります。そのための準備として、$U^{2^x}|\psi\rangle$が以下のように書けることに留意しておきます。

$$
\begin{aligned}
U^{2^x}|\psi\rangle&=U^{2^x-1}U|\psi\rangle\\
&=U^{2^x-1}e^{2\pi i\theta}|\psi\rangle\\
&=U^{2^x-2}e^{2\pi i\theta2}|\psi\rangle\\
&=\cdots\\
&=e^{2\pi i\theta2^x}|\psi\rangle
\end{aligned}
$$

この量子回路に対して、ステップ 1, 2, ... $n+1$での量子状態は以下のようになることが分かります。

- ステップ 1 : $\frac{1}{\sqrt{2^n}}(|0\rangle+|1\rangle)^{\otimes n}|\psi\rangle$
- ステップ 2 : $\frac{1}{\sqrt{2^n}}(|0\rangle+e^{2\pi i\theta2^{n-1}}|1\rangle)(|0\rangle+|1\rangle)^{\otimes n-1}|\psi\rangle$
- $\cdots$
- ステップ $n+1$ : $\frac{1}{\sqrt{2^n}}(|0\rangle+e^{2\pi i\theta2^{n-1}}|1\rangle)(|0\rangle+e^{2\pi i\theta2^{n-2}}|1\rangle)\cdots(|0\rangle+e^{2\pi i\theta2^0}|1\rangle)|\psi\rangle$

ステップ $n+1$後の$n$ビットレジスタの状態をよく見ると、この状態はQFTで$j$を$2^n\theta$としたものと同等であることが分かります。つまり、この$n$ビットレジスタに逆フーリエ変換$\rm{QFT}^\dagger$を施せば、状態$|2^n\theta\rangle$が得られることになります。この状態を測定することで、$2^n\theta$、つまり固有値の位相$\theta$（を$2^n$倍したもの）を求めることができるというのがQPEです（下図）。

<a id='qpe_nqubit_fig'></a>
```{image} figs/qpe.png
:alt: qpe
:class: bg-primary mb-1
:width: 700px
:align: center
```

### 確率分布の表現 <a id='algo_prob'></a>

まず、以下のような回路を考えてみます。Cはコンフィグレーション用レジスタと呼ぶもので、このレジスタを構成する量子ビットの配位によって、所望の確率分布を設定することができるようなレジスタのことです。このレジスタの構造は問題に依るので、どういう回路になるというのは一概には言えません。しかし、Cレジスタが$n$個の量子ビットを持つ場合、その計算基底

$$
|l\rangle_C\equiv\{|l\rangle\;|\;l=0,1,\cdots,N-1\}
$$

（$N=2^n$）が$p_l$に何らかの対応関係を持つものとします。


### 状態生成 <a id='algo_state'></a>

$d$個の量子ビットからなる作業用レジスタ$D$を別に用意して、状態を$|0\rangle_D\equiv|0\rangle^{\otimes d}$に初期化します。ここで

$$
U|l\rangle_C|0\rangle_D = |l\rangle_C|\tilde{p}_l\rangle_D
$$

のように作用するユニタリー演算$U$が存在するものと仮定してみます。$\tilde{p}_l$は$\tilde{p}_l=\lfloor2^dp_l\rfloor$で定義される量です。$|\tilde{p}_l\rangle_D$という状態は、Dレジスタのビット数$d$を$\max{\tilde{p}_l}<2^d$になるように選べば、$\tilde{p}_l=\sum_{k=0}^{d-1}\tilde{p}_l^{(k)}2^k$と2進数表記したときのビット列$\{\tilde{p}_l^{(k)}\in\{0,1\}|l=0,1,\cdots,M-1\}$（$M=2^d$）が作る計算基底になっているということが分かると思います。

これは重要な一歩ではあるのですが、後々の計算を簡単にするために、ここでは少し違う状態を生成することを考えます。この方法は参考文献[x]で飯山が提案したものです。ここで考えるのは、$p_l$そのものではなく

$$
\lambda_l=-\log_\gamma p_l
$$

です（$\gamma$は1より大きい任意の実数）。$0<p_l<1$なので、底$\gamma$を1に近づけていくほど$\lambda_l$は大きくなります。その時、$\tilde{\lambda}_l=\lfloor\lambda_l\rfloor$で与えられる整数$\tilde{\lambda}_l$を使って、$\tilde{p}_l=\gamma^{-\tilde{\lambda}_l}$と再定義することにします。

この$\tilde{\lambda}_l$を計算基底として表現する状態$|\tilde{\lambda}_l\rangle$をDレジスタに作るにはどうすればいいでしょうか。。これは上で学んだ量子位相推定（QPE）を使えば可能です。QPEでは、$U|\psi\rangle=e^{2\pi i\theta}|\psi\rangle$となるユニタリー演算$U$とその固有ベクトル$|\psi\rangle$を考えました。$|0\rangle$と$|1\rangle$を固有ベクトルとして持つとして、その状態への作用を考えれば、この演算はそれぞれ固有値1と$e^{2\pi i\theta}$を持つ位相変換に対応していることが分かるでしょう。つまり、

$$
U=\begin{bmatrix}1&0\\0&e^{2\pi i\theta}\end{bmatrix}
$$

です。QPEはこの$U$をビットごとに2の冪乗回適用していくことで、測定前の状態として$|2^n\theta\rangle$を得ることができました。同じ理由で、固有ベクトル$|1\rangle$に対して、$\theta_l=\tilde{\lambda}_l/2^n$として

$$
U_\lambda=\begin{bmatrix}1&0\\0&e^{2\pi i\theta_l}\end{bmatrix}
$$

となるユニタリー$U_\lambda$を適用すれば、状態として$|\tilde{\lambda}_l\rangle$を得ることができることが分かりますね。

状態生成の回路をまとめると、以下のようになります。



### 求める振幅の生成 <a id='algo_amplitude'></a>

次のステップは、上で得られた状態$|\tilde{\lambda}_l\rangle$から、$\tilde{\lambda}_l$を振幅に持つような状態を生成することです。$\lambda_l=-\log_\gamma p_l$のような変換を行った理由は、ここで明らかになります。

$\tilde{\lambda}_l$は$d$ビットの整数として表現されていることを思い出すと、上で$\tilde{p}_l$にやったのと同様に

$$
\tilde{\lambda}_l=\sum_{k=0}^{d-1}\tilde{\lambda}_l^{(k)}2^k
$$

と2進数で表記できます。こうすることで、Dレジスタに入っている$|\tilde{\lambda}_l\rangle_D$の各ビット（$k$番目のビットは$\tilde{\lambda}_l^{(k)}$）を制御ビットとする制御ゲートを考えれば、$\tilde{\lambda}_l^{(k)}$を別のレジスタに移すという方法が見えてきます。
そこで、$d$量子ビットを持つ別のレジスタEを考えて、そこに制御$R_y$ゲートで$\tilde{\lambda}_l$の情報を移すことを考えてみます。$Y$軸周りの回転ゲート$R_y$は

$$
R_y(\phi)=\begin{bmatrix}\cos\frac\phi2&-\sin\frac\phi2\\\sin\frac\phi2&\cos\frac\phi2\end{bmatrix}
$$

ですね（$\phi$は回転角）。まずEレジスタを$|0\rangle$に初期化して、$|0\rangle_E\equiv|0\rangle^{\otimes d}$とします。Dレジスタが$|\tilde{\lambda}_l\rangle_D$の時に、Dレジスタの$k$番目量子ビットで制御された$R_y(2\phi)$ゲートをEレジスタの対応する量子ビットに適用すると

$$
R_y(2\phi_k)|\tilde{\lambda}_l\rangle_D|0\rangle_E=\bigotimes_{k=0}^{d-1}|\tilde{\lambda}_l\rangle_D \left[(\cos\phi_k)^{\tilde{\lambda}_l^{(k)}}|0\rangle_k+(\sin\phi_k)^{\tilde{\lambda}_l^{(k)}}|1\rangle_k\right]_E
$$

と書くことができます（$\tilde{\lambda}_l^{(k)}$の値によって$\cos\phi_k$や$\sin\phi_k)$が位相にかかるかどうか決まる）。Eレジスタの量子ビットが$|1\rangle$の項は今は興味がないものとして、まとめて$|w\rangle$としておきます（Eレジスタ全体で$|1\rangle$を含む場合は、大文字の$|W\rangle$としておきます）。とすると、

$$
\begin{aligned}
R_y(2\phi_k)|\tilde{\lambda}_l\rangle_D|0\rangle_E&=\bigotimes_{k=0}^{d-1}|\tilde{\lambda}_l\rangle_D \left[(\cos\phi_k)^{\tilde{\lambda}_l^{(k)}}|0\rangle_k+|w\rangle_k\right]_E \\
&=\bigotimes_{k=0}^{d-1}|\tilde{\lambda}_l\rangle_D \left[\gamma^{-\tilde{\lambda}_l^{(k)}2^k}|0\rangle_k+|w\rangle_k\right]_E \\
&=|\tilde{\lambda}_l\rangle_D \left[\gamma^{-\sum_{k=0}^{d-1}\tilde{\lambda}_l^{(k)}2^k}|0\rangle_k+|W\rangle\right]_E \\
&=\gamma^{-\tilde{\lambda}_l}|\tilde{\lambda}_l\rangle_D|0\rangle_E+|W\rangle_{D\otimes E} \\
&=\tilde{p}_l|\tilde{\lambda}_l\rangle_D|0\rangle_E+|W\rangle_{D\otimes E} \\
\end{aligned}
$$

この状態に$R_y(\phi)$を適用すると




### 振幅の増幅 <a id='algo_amplitude_amplification'></a>
